## Parameter identification for various models

To investigate the performance of parameter identification for different electrochemical models we will start with synthetic data from the highest order model in PyBOP (Many-particle DFN) and try to identify the correct parameter values on the reduced order models.

### Setting up the Environment

Before we begin, we need to ensure that we have all the necessary tools. We will install PyBOP and upgrade dependencies:

In [ ]:
%pip install --upgrade pip ipywidgets pybamm -q
%pip install pybop -q

/home/nicola/GitHub/PyBOP/.nox/notebooks-overwrite/bin/python3: No module named pip


Note: you may need to restart the kernel to use updated packages.
/home/nicola/GitHub/PyBOP/.nox/notebooks-overwrite/bin/python3: No module named pip


Note: you may need to restart the kernel to use updated packages.


Next, we import the added packages plus any additional dependencies,

In [ ]:
import numpy as np
import pybamm

import pybop

go = pybop.plot.PlotlyManager().go
pybop.plot.PlotlyManager().pio.renderers.default = "notebook_connected"

Let's fix the random seed in order to generate consistent output during development, although this does not need to be done in practice.

In [ ]:
np.random.seed(8)

## Optimising the Parameters

First, we define the model to be used for the parameter optimisation,

In [ ]:
synth_model = pybamm.lithium_ion.DFN(options={"particle size": "distribution"})
parameter_values = pybamm.ParameterValues("Chen2020")
parameter_values = pybamm.get_size_distribution_parameters(parameter_values)
parameter_values.set_initial_state(0.5);

### Simulating Forward Model

We can then simulate the model using the `predict` method, with a default constant current to generate voltage data.

In [ ]:
n_points = 350
t_eval = np.linspace(0, 1600 + 1000, n_points)
current = np.concatenate(
    [np.ones(100) * parameter_values["Nominal cell capacity [A.h]"], np.zeros(250)]
)
parameter_values["Current function [A]"] = pybamm.Interpolant(t_eval, current, pybamm.t)

In [ ]:
solution = pybamm.Simulation(synth_model, parameter_values=parameter_values).solve(
    t_eval=t_eval
)

### Adding Noise to Voltage Data

To make the parameter estimation more realistic, we add Gaussian noise to the data.

In [ ]:
sigma = 0.001
corrupt_values = solution["Voltage [V]"](t_eval) + np.random.normal(
    0, sigma, len(t_eval)
)
go.Figure(
    data=go.Scatter(x=t_eval, y=corrupt_values, mode="lines"),
    layout=go.Layout(title="Corrupted Voltage", width=800, height=600),
)

## Identifying the Parameters

We will now set up the parameter estimation process by defining the datasets for optimisation and selecting the model parameters we wish to estimate.

### Creating a Dataset

The dataset for optimisation is composed of time, current, and the noisy voltage data:

In [ ]:
dataset = pybop.Dataset(
    {
        "Time [s]": t_eval,
        "Current function [A]": current,
        "Voltage [V]": corrupt_values,
    }
)

Next, we define the model parameters for optimisation. Furthermore, PyBOP provides functionality to define a distribution for the parameters. The initial parameter values used in the optimisation will be randomly drawn from the distribution.

In [ ]:
parameter_values.update(
    {
        "Positive electrode thickness [m]": pybop.ParameterDistribution(
            pybop.Gaussian(7.56e-05, 0.05e-05, truncated_at=[65e-06, 85e-06]),
        ),
        "Negative electrode thickness [m]": pybop.ParameterDistribution(
            pybop.Gaussian(8.52e-05, 0.05e-05, truncated_at=[75e-06, 95e-06]),
        ),
    }
)

We can now define the output signal, the problem (which combines the model with the dataset) and construct a cost function which in this example is the `GravimetricEnergyDensity()` used to maximise the gravimetric energy density of the cell.

In [ ]:
models = [
    pybamm.lithium_ion.SPM(),
    pybamm.lithium_ion.SPMe(),
]

Let's construct PyBOP's optimisation class for each model. This class provides the methods needed to fit the forward model. For this example, we use an evolution strategy (XNES) as the optimiser.

In [ ]:
optims = []
xs = []
for model in models:
    print(f"Running {model.name}")
    simulator = pybop.pybamm.Simulator(
        model, parameter_values=parameter_values, protocol=dataset
    )
    cost = pybop.SumSquaredError(dataset)
    problem = pybop.Problem(simulator, cost)
    options = pybop.PintsOptions(
        verbose=True,
        max_iterations=60,
        max_unchanged_iterations=10,
    )
    optim = pybop.XNES(problem, options=options)
    optim.set_population_size(5)
    result = optim.run()
    optims.append(optim)
    xs.append(result.x)

Running Single Particle Model


| Iter: 1 | Evals: 5| Best Parameters: [8.46950974e-05 7.56852452e-05] | Best Cost: 0.3646285829033633


| Iter: 2 | Evals: 10| Best Parameters: [8.45576205e-05 7.48482936e-05] | Best Cost: 0.34295023553666765


| Iter: 3 | Evals: 15| Best Parameters: [8.40822845e-05 7.47418916e-05] | Best Cost: 0.3326103718627962


| Iter: 4 | Evals: 20| Best Parameters: [8.36443742e-05 7.38270215e-05] | Best Cost: 0.3064398478281154


...

| Iter: 7 | Evals: 35| Best Parameters: [8.20939817e-05 7.50084524e-05] | Best Cost: 0.005186011597850185


| Iter: 8 | Evals: 40| Best Parameters: [8.18439586e-05 7.49378232e-05] | Best Cost: 0.004505286430878077


| Iter: 9 | Evals: 45| Best Parameters: [8.14885675e-05 7.48549822e-05] | Best Cost: 0.0037824283364975867


| Iter: 10 | Evals: 50| Best Parameters: [8.09223410e-05 7.48631074e-05] | Best Cost: 0.003207219527061374


OptimisationResult:
  Best result from 1 run(s).
  Initial parameters: [8.58310889e-05 7.70415062e-05]
  Optimised parameters: [8.06724438e-05 7.46468378e-05]
  Best cost: 0.003028980393277891
  Optimisation time: 52.294625997543335 seconds
  Number of iterations: 21
  Number of evaluations: 106
  Reason for stopping: No significant change for 10 iterations.


In [ ]:
for optim, x in zip(optims, xs, strict=False):
    print(f"| Model: {optim.problem.simulator.model.name} | Results: {x} |")

| Model: Single Particle Model | Results: [7.76386911e-05 6.84139111e-05] |
| Model: Single Particle Model with electrolyte | Results: [8.06724438e-05 7.46468378e-05] |


## Plotting and Visualisation

PyBOP provides various plotting utilities to visualise the results of the optimisation.

### Comparing System Response

We can quickly plot the system's response using the estimated parameters compared to the initial parameters:


In [ ]:
for optim, x in zip(optims, xs, strict=False):
    pybop.plot.problem(
        optim.problem,
        problem_inputs=x,
        title=optim.problem.simulator.model.name,
    )

### Cost Landscape

Finally, we can visualise the cost landscape and the path taken by the optimiser:


In [ ]:
for optim in optims:
    result.plot_surface(title=optim.problem.simulator.model.name)